In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yaifinal/test_covid.csv
/kaggle/input/yaifinal/SampleSubmission.csv
/kaggle/input/yaifinal/train_covid.csv


In [127]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader


# Read the train/test file from kaggle
train = pd.read_csv('../input/yaifinal/train_covid.csv')
test = pd.read_csv('../input/yaifinal/test_covid.csv')

0        NaN
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
14387    NaN
14388    NaN
14389    NaN
14390    NaN
14391    NaN
Name: stringency_index, Length: 14392, dtype: float64


In [128]:
# Fill missing value
# fill empty stringency_index to zero
train["stringency_index"] = train["stringency_index"].fillna(0)
test["stringency_index"] = test["stringency_index"].fillna(0)

# fill other empty space to median
train = train.fillna(train.median())
test = test.fillna(train.median())

# normalize data by its category location
for key in tqdm(sorted(set(train["location"]))):
    
    temp = train[train["location"] == key]
    temp2 = test[test["location"] == key]    
    
    temp["stringency_index"] = temp["stringency_index"] / train["stringency_index"].mean()
    temp["population"] = temp["population"] / train["population"].mean()
    temp2["stringency_index"] = temp2["stringency_index"] / test["stringency_index"].mean()
    temp2["population"] = temp2["population"] / test["population"].mean()
    
    train[train["location"] == key] = temp
    test[test["location"] == key] = temp2

  0%|          | 0/208 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [129]:
## 3layer FC layer NN
class myModel(torch.nn.Module):
    def __init__(self):
        super(myModel, self).__init__()
        self.model = torch.nn.Sequential(
                        torch.nn.Linear(14*4, 7*4),
                        torch.nn.GELU(),
                        torch.nn.Linear(7*4, 4*4),
                        torch.nn.ReLU()                        ,
                        torch.nn.Linear(4*4, 1*4),
                        torch.nn.ReLU()
        )
        # [c*14, d*14, e*14, f*14] --> [c, d, e, f]
        
    def forward(self, x):
        c_and_d = self.model(x) # get output
        c = c_and_d[:, 0] # new_case
        d = c_and_d[:, 1] # new_death
        e = c_and_d[:, 2] # stringency_index
        f = c_and_d[:, 3] # population
        return c, d, e, f

class myDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {"inputs": self.inputs[idx], "outputs": self.outputs[idx]}
    
def my_collate(samples):
    inputs = []
    outputs = []
    for sample in samples:
        inputs.append(sample["inputs"])
        outputs.append(sample["outputs"])
        
    return {"inputs": torch.tensor(inputs), "outputs": torch.tensor(outputs)}

In [130]:
# initialize the dataset 
train_case = [0.] * 14
train_death = [0.] * 14
train_stringency_index = [0.] * 14
train_population = [0.] * 14
train_input = train_case + train_death + train_stringency_index + train_population
train_inputs = []
train_outputs = []
locations = []

test_case = [0.] * 14
test_death = [0.] * 14
test_stringency_index = [0.] * 14
test_population = [0.] * 14

test_input = test_case + test_death + test_stringency_index + test_population
test_inputs = []

In [131]:
# Make dataset for training 
# [c*14, d*14, e*14, f*14] --> [c, d, e, f]
# (ex: given: 4/1~4/14 case & death, predict: 4/15 case & death)
for key in tqdm(sorted(set(train["location"]))):
    locations.append(key)
    temp_df = train[train["location"] == key]
    train_case = [0.] * 14
    train_death = [0.] * 14
    train_stringency_index = [0.] * 14
    train_population = [0.] * 14
    train_input = train_case + train_death + train_stringency_index + train_population
    train_inputs.append(train_input)
    
    append_flag = 1
    for index, row in temp_df.iterrows():
        if(append_flag):
            train_outputs.append([float(row["new_cases"]), float(row["new_deaths"]), float(row["stringency_index"]), float(row["population"])])
            append_flag = 0
            
        train_case = train_case[1:] + [float(row["new_cases"])]
        train_death = train_death[1:] + [float(row["new_deaths"])]
        train_stringency_index = train_stringency_index[1:] + [float(row["stringency_index"])]
        train_population = train_population[1:] + [float(row["population"])]
        train_input = train_case + train_death + train_stringency_index + train_population
        #train_inputs.append(train_input)
        
        # To handle too many multiple values & the last value
        # (ex: [0, 0, .... , 0, 0] --> [0, 0])
        if train_input in train_inputs or index == temp_df["id"].values[-1]: continue
        else: train_inputs.append(train_input); append_flag = 1

100%|██████████| 208/208 [00:15<00:00, 13.53it/s]


In [132]:
# Evaluation dataset
test_input_dict = {}
for key in tqdm(sorted(set(train["location"]))):
    
    # To predict 5/1, train 4/17~4/30 
    temp_df = train[train["location"] == key]
    temp_df = temp_df[(temp_df['date'] >= '2020-04-17') & (temp_df['date'] <= '2020-04-30')]
    
    
    test_case = list(map(lambda x: float(x), temp_df["new_cases"].values))
    test_death = list(map(lambda x: float(x), temp_df["new_deaths"].values))
    test_stringency_index = list(map(lambda x: float(x), temp_df["stringency_index"].values))
    test_population = list(map(lambda x: float(x), temp_df["population"].values))
    
    if(len(test_case) < 14): test_case = [0.] * (14 - len(test_case)) + test_case
    if(len(test_death) < 14): test_death = [0.] * (14 - len(test_death)) + test_death
    if(len(test_stringency_index) < 14): test_stringency_index = [0.] * (14 - len(test_stringency_index)) + test_stringency_index    
    if(len(test_population) < 14): test_population = [0.] * (14 - len(test_population)) + test_population      
    test_input = test_case + test_death + test_stringency_index + test_population
    
    test_input_dict[key] = test_input

100%|██████████| 208/208 [00:00<00:00, 330.17it/s]


In [133]:
batch_size = 10

## Dataset Load
dataset = myDataset(train_inputs, train_outputs)
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers':0,
          'drop_last': True,
          'collate_fn': my_collate
         }
train_generator = DataLoader(dataset, **params)

## Model load
model = myModel()
# Loss/Optimizer define
optimizer = torch.optim.AdamW(model.parameters(), amsgrad = True)

class RMSLELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))
    
# Loss
criterion = RMSLELoss()

In [134]:
# Training
model.train()
for epoch in tqdm(range(100)):
    for k in train_generator:
        pred_c, pred_d, pred_e, pred_f = model(k["inputs"])        
        
        loss_c = criterion(pred_c, k["outputs"][:, 0])
        loss_d = criterion(pred_d, k["outputs"][:, 1])
        loss_e = criterion(pred_e, k["outputs"][:, 2])
        loss_f = criterion(pred_f, k["outputs"][:, 3])
        total_loss = (loss_c + loss_d + loss_e + loss_f) / 4              
        optimizer.zero_grad() # initialize gradient vector to zero
        total_loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 1)
        optimizer.step()
        
    print("epoch: ", total_loss)

  1%|          | 1/100 [00:02<04:25,  2.68s/it]

epoch:  tensor(0.6051, grad_fn=<DivBackward0>)


  2%|▏         | 2/100 [00:05<04:23,  2.69s/it]

epoch:  tensor(0.4422, grad_fn=<DivBackward0>)


  3%|▎         | 3/100 [00:07<04:15,  2.64s/it]

...

epoch:  tensor(0.8975, grad_fn=<DivBackward0>)


 99%|█████████▉| 99/100 [04:10<00:02,  2.53s/it]

epoch:  tensor(0.7008, grad_fn=<DivBackward0>)


100%|██████████| 100/100 [04:12<00:00,  2.53s/it]

epoch:  tensor(0.5999, grad_fn=<DivBackward0>)


In [135]:
## Evaluation
eval_location = []

# save all predictions
predict_case = []
predict_death = []

model.eval()
for idx, row in test.iterrows():
    if(row["location"] not in eval_location):
        #print(row["location"])
        eval_location.append(row["location"])
        eval_input = test_input_dict[row["location"]]           
    c, d, e, f = model(torch.tensor(eval_input).unsqueeze(0))
    predict_case.append(c)
    predict_death.append(d)
    
    # update the last new_case/new_death value
    eval_input[13] = c
    eval_input[27] = d
    eval_input[41] = e
    eval_input[55] = f

In [136]:
# Make submission file
predict_all = torch.stack([torch.tensor(predict_case), torch.tensor(predict_death)], dim = 1)
ppredict = predict_all.numpy()
ppredict = pd.DataFrame(ppredict, columns = ["new_cases", "new_deaths"])
ppredict["id"] = ppredict.index

ppredict = ppredict[["id", "new_cases", "new_deaths"]]

In [137]:
# Save submission file
ppredict.to_csv("Submit.csv", index = False)